<div style='background-color:#f7f7f7; padding-top:30px; padding-left:20px; padding-right:20px; padding-bottom:30px'>
    <center>
        <div style='  display: block;
  font-size: 2em;
  font-weight: bold;  display: block;
  font-size: 2em;
  font-weight: bold;'>LCP-B - Processing of SEVN data for binary black holes mass distribution analysis
        </div>
    <center>
        <br>
    <i>Tommaso Bertola, Giacomo Di Prima, Giuseppe Viterbo, Marco Zenari</i></center>
    <center>
    <i>All authors contributed equally to the project</i></center>
    

# The physical problem
The subject of our studies are binary black holes obtained by the population synthesis code SEVN, in particular we aim to understand the importance of the input features of the program in guiding the evolution, and gaining some insight about the final mass distribution, and some correlated quantities.

1. Chirp mass and q
Describe


But firstly it is important to understand what exatly are black holes, and how stars evolve into them. 
When a star ends its nuclear burning material, no sorce of energy can support the gravity pressure and the star goes out of the hydrostatic equilibrium (described as the balance between Pressior and Nuclear forces and Gravity) and the core collapse begins. In particular, if the Mass of the star is bigger than 20 M_sun the gravity is so strong that no source of pressure can stop the core collpase, and a black hole is form. 
As we know from observations, almost all the massive stars (M > 20 Msun) are in binaries (or higher multiplicity systems), so it is important to understand the evolution of binary system.


2. How do supernova deviate our predictions
The core collapse is one fondamental ingredient to define to the final distribution of the mass of a black hole. During the final stages of evolution the Silicon burning shell ads material to the Iron core, and when this one gets to a total mass bigger than the Chandrasekar Mass (1.3-1.7 M_sun), the degenerate electron pressure is not able to sostain anymore the gravity, and so the runaway core collapse begins. -insert equation- 
When the shock wave is not stopped the star explode as a SN, leaving a compact remant, either a NS or a low-mass BH (is it true? we should check if the Direct Collapse produce more massive black holes maybe). When such an event occur, asymmetry in mass ejection or in the neutrino emission (cool down the shockwave generally), or also symmetric mass loss in binary system, leads to the so called Kicks, which is the gained momentum obtained by the compact object after the SN. Unfortunalty for black holes supernova kicks there is no real observation (model selected by seven), and the kick is extracte from a model dependent proability distribution -our simulation where from Freyer 2012, need to check-. In our algorithm we try to derive how much this kind of phenomena is able to deviate our prediction of the output mass distribution.


3. How do the evolution in a binary system interfere with the evolution?
For the best predicted records we also investigate the history evolution, focusing on the mass exchange channels RLO and CE. 



For this reason we examin the following parameter space:

|Parameter type| Tested values| 
|:----|:----|
| [M_0, M_1]  | Initial masses of the stars in the binary   |
| Z         | Metallicity of the two starts in the binary |
| a         | semi-major axis of the binary               |
| e         | eccentricity of the binary                  |




# Our aim

# A brief recap on the preprocessing of the data
Given the initial 1.7TB of data, we need to first preprocess it in order to extract only the meaningful information.
To achieve this goal we use a cluster of 6 VMs hosted in CloudVeneto leveraging the capabilities of Python Dask library.

We show below the implementation of the code used to parse the whole dataset. The data extracted from this algorithm is then saved onto disk to avoid rerunning the whole computation which takes about 2 hours to complete.

In [ ]:
# output_{}.csv
output_column_to_read = ['name', 'Mass_0', 'RemnantType_0',
                         'Mass_1', 'RemnantType_1',
                         'Semimajor','Eccentricity',
                         'GWtime','BWorldtime']
output_column_type = ['string', 'float64', 'int64',
                      'float64', 'int64',
                      'float64', 'float64',
                      'float64', 'float64']

# evolved_{}.dat
evolved_column_to_read = ['name', 'Mass_0',
                          'Z_0', 'SN_0',
                          'Mass_1', 'SN_1',
                          'a', 'e']
evolved_column_type = ['string', 'float64',
                       'float64', 'string',
                       'float64', 'string',
                       'float64', 'float64']

# further columns to remove at the end 
drop_list = ['RemnantType_0',  'RemnantType_1']

In [ ]:
#OUTPUT files processing
output = pd.read_csv(paths[0],                              # read the file
                     usecols=output_column_to_read,         # read only some cols
                     dtype=dict(zip(output_column_to_read,  # specify the types
                                    output_column_type))).\ #
            rename(columns={'Mass_0':'Mass_0_out',          # rename columns
                            'Mass_1':'Mass_1_out'})         #

# mask to select only the black holes binaries, defined by RemnantType
idxBHBH=(output.RemnantType_0==6) & (output.RemnantType_1==6) & (output.Semimajor.notnull())

# apply the mask
output=output[idxBHBH]

In [ ]:
#EVOLVED files processing

#extracting the alpha parameter from the path of the file 
alpha = float(re.findall(r".+(?<=A)(.*)(?=L)",
                         paths[1])[0])

#read the columns we are interested in from the evolved file
evolved = pd.read_table(paths[1],                               # read file
                        sep='\s+',                              # separate by spaces
                        usecols=evolved_column_to_read,         # read only some columns
                        dtype=dict(zip(evolved_column_to_read,  # specify the types
                                       evolved_column_type)))   #
#NB: sep='\s+' is needed because there are different number of spaces separareting the columns

#adding the column with the alpha parameter
evolved['alpha'] = alpha

In [ ]:
#LOGFILE files processing

logfile = pd.read_csv(paths[2],    # read the file
                      header=None) # there is no header

In [ ]:
#Running Regex on the line of the logfile to extrac useful informations
df_RLO = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);RLO_BEGIN;").\  # searching for string "RLO_BEGIN"
            dropna().\                                                    # dropping nan
            rename(columns={0:'name', 1:'ID'}).\                          # rename columns
            groupby(['name']).\                                           # grouping by name
            size().\                                                      # and counting the number of RLO
            to_frame(name='RLO').\                                        # 
            reset_index()                                                 # to have a nice dataframe


df_CE = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);CE;").\  # searching for string "CE"
            dropna().\                                            # dropping nan
            rename(columns={0:'name', 1:'ID'}).\                  # rename
            groupby(['name']).\                                   # grouping by name
            size().\                                              # 
            to_frame(name='CE').\                                 # and counting the number of CE
            reset_index()                                         # to have a nice dataframe


df_BSN = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);BSN;").\  #searching for string "BSN"
            dropna().\                                              # dropping nan
            rename(columns={0:'name', 1:'ID'}).\                    #rename
            groupby(['name']).\                                     #grouping by name
            size().\                                                #
            to_frame(name='BSN').\                                  #and counting the number of BSN
            reset_index()                                           #to have a nice dataframe

df_No_Kick = logfile[0].str.extract(r"S;((?:\d*\_)?\d+);(\d+);SN;.+:(0):.+:.+:.+.").\ #searching for string "No_Kick"
            dropna().\                                                                # dropping nan
            rename(columns={0:'name', 1:'ID', 2: 'No_Kick'}).\                        #rename
            groupby(['name']).\                                                       #grouping by name
            size().\                                                                  #
            to_frame(name='No_Kick').\                                                #and counting the number of "No_Kick"
            reset_index()                                                             #to have a nice dataframe

In [ ]:
#MERGE
bhbh = evolved.merge(output, on=['name'], how='inner').\    #innerg join on the name between evolved and output
               merge(df_RLO, on=['name'], how='left').\     #left join on the name with df_RLO
               merge(df_CE,  on=['name'], how='left').\     #left join on the name with df_CE
               merge(df_BSN, on=['name'], how='left').\     #left join on the name with df_BSN
               merge(df_No_Kick, on=['name'], how='left').\ #final join on the name with No_Kicks
               fillna(value=0).\                            #setting nan to zero
               drop(columns=drop_list)                      #dropping no longer useful columms

In [ ]:
#Adding some columns with physical meaning
bhbh['tdelay'] = bhbh['GWtime'] + bhbh['BWorldtime'] #time delay

#defining the max mass of output
bhbh['Mass_max_out'] = bhbh['Mass_1_out']
bhbh['Mass_max_out'] = bhbh['Mass_max_out'].\
                        where(cond=(bhbh['Mass_max_out'] > bhbh['Mass_0_out']),
                              other=bhbh['Mass_0_out'])

#defining q=m1/m2 with m2>,m1
bhbh['q'] = bhbh['Mass_1_out']/bhbh['Mass_0_out']
bhbh['q'] = bhbh['q'].\
            where(cond=(bhbh['Mass_1_out'] < bhbh['Mass_0_out']),
                  other=bhbh['Mass_0_out']/bhbh['Mass_1_out'])

#defining the Chirp mass
bhbh['Mass_chirp'] = ((bhbh['Mass_0_out'] * bhbh['Mass_1_out'])**(3/5))/((bhbh['Mass_0_out'] + bhbh['Mass_1_out'])**(1/5))

# A recap on Machine Learning approaches used
## We discretize the q, ratio between masses

## XGBoost our Lord and Saviour

# Grid search
## Accuracies used
## What varies
## What improves

# Discussion on the predictions
The algorithm is able to perform predictions either on the `q` or the `chirp mass`, as its implementation barely varies.

Here we are showing the results of the predictions made with `q`. In the following section we are showing the same results but run with the `chrip mass` 

## Analysis with the `q`
### Feature importance
A first analysis of the performance of the model is carried out through the feature importance.
This analysis shows how much each of the features of the training dataset is used to split the data in the XGBoost architecure.
Other similar importance metrics such as the gain, which is the average of the gain per split when a specific feature was used, show that `Mass_0`, `Z_0` the metallicity and `Mass_1` were the most useful and could better help the training process.
We remind the feature importance does not take into account the test set, as it is just another way of displaying the information learnt dirung the training.

<img src='./notebooks/Figures/importance_q_yeskicks_weight.png'>
<img src='./notebooks/Figures/importance_q_yeskicks_gain.png'>
<img src='./notebooks/Figures/importance_q_yeskicks_total_gain.png'>

### Confusion matrix
To check by eye how the algorithm performs on the unseen test set, we plot the confusion matrix of the predictions.

<img src='notebooks/Figures/lcp/confusion_matrix_labels_yeskicks.png'>

Each square of the picture shows the number of counts that were assigned to a specific label, given the value of the true label.
It is immediate to spot higher countings of binary systems that are in accordance with their true label, thus suggesting that the model really understood the hidden processes.
Finally it is easy to spot how higher labels tend to be more frequently misclassified, but this is easily explained by the unbalanced dataset we started with.
Very few binary systems of actual lower q labels happen to be missclassified with high labels as shown by the white patch in the upper left corner.

### Comparison of the empirical and predicted `q` label distributions
A different and more comprehensive visualization to better check the performances of the model is to plot the distributions of the empirical and true labels, as shown in the following plot.

<img src='notebooks/Figures/lcp/hist_q_yes_kicks.png'>

The distributions seem to match very well, in all the varying regions of the q labels.
The model seems to capture, but only to a certain extent, the bump around the region of labels 60 to 70 as there are some deviations from the smooth trend.
There seems to be a slight overcouting for the smaller labels compared to the true distribution, but as can be seen from the Kullback-Liebler and Jensen-Shannon metrics, this does not inflence the overall correspondance between the two distributions.

## Analysis with the `chirp mass`

# Final consideration and conclusions 